In [158]:
import pandas as pd

In [159]:
df=pd.read_csv("Downloads/historical_data.csv")

In [160]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845.0,american,1.0,4.0,3441.0,4.0,557.0,1239.0,33.0,14.0,21.0,446.0,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477.0,mexican,2.0,1.0,1900.0,1.0,1400.0,1400.0,1.0,2.0,2.0,446.0,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477.0,NaN,1.0,1.0,1900.0,1.0,1900.0,1900.0,1.0,0.0,0.0,446.0,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477.0,NaN,1.0,6.0,6900.0,5.0,600.0,1800.0,1.0,1.0,2.0,446.0,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477.0,NaN,1.0,3.0,3900.0,3.0,1100.0,1600.0,6.0,6.0,9.0,446.0,650.0


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10589 entries, 0 to 10588
Data columns (total 16 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   market_id                                     10536 non-null  float64
 1   created_at                                    10589 non-null  object 
 2   actual_delivery_time                          10586 non-null  object 
 3   store_id                                      10588 non-null  float64
 4   store_primary_category                        10296 non-null  object 
 5   order_protocol                                10535 non-null  float64
 6   total_items                                   10588 non-null  float64
 7   subtotal                                      10588 non-null  float64
 8   num_distinct_items                            10588 non-null  float64
 9   min_item_price                                10588 non-null 

In [162]:
# Unique restaurants
unique_stores = df['store_id'].unique()
print(f"Number of unique restaurants: {len(unique_stores)}")


Number of unique restaurants: 431


In [163]:
# Unique markets / cities
unique_markets = df['market_id'].unique()
print(f"Number of unique markets: {len(unique_markets)}")

Number of unique markets: 7


In [164]:
import numpy as np
num_users = 5000
np.random.seed(42)  # reproducible

users = pd.DataFrame({
    'user_id': range(1, num_users+1),
    'signup_date': pd.to_datetime(
        np.random.choice(pd.date_range(start='2022-01-01', end='2025-01-01'), num_users)
    ),
    'city': np.random.choice(unique_markets, num_users),
    'device_type': np.random.choice(['iOS', 'Android', 'Web'], num_users),
    'is_active_flag': np.random.choice([0, 1], num_users, p=[0.2, 0.8])
})

users.head()


,user_id,signup_date,city,device_type,is_active_flag
0,1,2024-05-10,2.0,iOS,0
1,2,2024-12-31,6.0,iOS,1
2,3,2024-11-10,2.0,Android,1
3,4,2022-05-02,2.0,iOS,1
4,5,2023-04-12,1.0,Web,1


In [165]:
num_dashers = 500

dashers = pd.DataFrame({
    'dasher_id': range(1, num_dashers+1),
    'city': np.random.choice(unique_markets, num_dashers),
    'rating': np.round(np.random.uniform(3.0, 5.0, num_dashers), 1),
    'tenure_days': np.random.randint(30, 1000, num_dashers),
    'total_completed_orders': np.random.randint(50, 5000, num_dashers)
})

dashers.head()


,dasher_id,city,rating,tenure_days,total_completed_orders
0,1,2.0,3.6,993,2405
1,2,4.0,3.4,588,2556
2,3,2.0,4.1,275,383
3,4,4.0,3.9,532,2612
4,5,4.0,3.8,118,2770


In [166]:
restaurants = df[['store_id', 'store_primary_category', 'market_id']].drop_duplicates().copy()
restaurants.rename(columns={'store_primary_category':'cuisine_type', 'market_id':'city'}, inplace=True)

# Add synthetic features
restaurants['avg_prep_time'] = np.random.randint(10, 45, len(restaurants))  # in minutes
restaurants['rating'] = np.round(np.random.uniform(3.0, 5.0, len(restaurants)), 1)

restaurants.head()


,store_id,cuisine_type,city,avg_prep_time,rating
0,1845.0,american,1.0,37,4.7
1,5477.0,mexican,2.0,27,4.1
2,5477.0,NaN,3.0,19,3.1
8,5477.0,indian,2.0,41,4.9
14,2841.0,italian,1.0,27,4.7


In [167]:
dashers.to_csv('doordash-analytics-project/data/synthetic/dashers.csv', index=False)

In [168]:
restaurants.to_csv('doordash-analytics-project/data/synthetic/restaurants.csv', index=False)

In [169]:
#identifying how many rows have broken data

df[['created_at', 'actual_delivery_time']].isna().sum()

created_at              0
actual_delivery_time    3
dtype: int64

In [170]:
#dropping the invalid rows

df = df.dropna(subset=['created_at', 'actual_delivery_time'])

In [171]:
df.copy()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845.0,american,1.0,4.0,3441.0,4.0,557.0,1239.0,33.0,14.0,21.0,446.0,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477.0,mexican,2.0,1.0,1900.0,1.0,1400.0,1400.0,1.0,2.0,2.0,446.0,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477.0,NaN,1.0,1.0,1900.0,1.0,1900.0,1900.0,1.0,0.0,0.0,446.0,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477.0,NaN,1.0,6.0,6900.0,5.0,600.0,1800.0,1.0,1.0,2.0,446.0,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477.0,NaN,1.0,3.0,3900.0,3.0,1100.0,1600.0,6.0,6.0,9.0,446.0,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10583,5.0,2015-02-15 04:35:55,2015-02-15 05:31:37,410.0,middle-eastern,5.0,3.0,4600.0,3.0,1100.0,1900.0,24.0,24.0,39.0,251.0,597.0
10584,5.0,2015-02-04 19:53:45,2015-02-04 20:54:06,410.0,middle-eastern,5.0,8.0,6200.0,6.0,300.0,1700.0,20.0,17.0,26.0,251.0,507.0
10585,5.0,2015-01-21 20:05:24,2015-01-21 20:29:36,410.0,middle-eastern,5.0,2.0,2800.0,1.0,1000.0,1000.0,31.0,8.0,11.0,251.0,484.0
10586,5.0,2015-02-14 01:33:26,2015-02-14 02:33:17,410.0,middle-eastern,5.0,3.0,4000.0,3.0,800.0,1900.0,24.0,10.0,10.0,251.0,656.0


In [172]:
#converting timestamps to datetime format and coercing bad values to null 

df.loc[:, 'created_at'] = pd.to_datetime(
    df['created_at'].astype(str).str.strip(),
    errors='coerce',
    format='mixed',
    utc=True
)

df.loc[:, 'actual_delivery_time'] = pd.to_datetime(
    df['actual_delivery_time'].astype(str).str.strip(),
    errors='coerce',
    format='mixed',
    utc=True
)

In [173]:
print(type(df.loc[0, 'created_at']), df.loc[0, 'created_at'])
print(df['created_at'].head(5).tolist())

<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2015-02-06 22:24:17+00:00
[Timestamp('2015-02-06 22:24:17+0000', tz='UTC'), Timestamp('2015-02-10 21:49:25+0000', tz='UTC'), Timestamp('2015-01-22 20:39:28+0000', tz='UTC'), Timestamp('2015-02-03 21:21:45+0000', tz='UTC'), Timestamp('2015-02-15 02:40:36+0000', tz='UTC')]


In [174]:
print(type(df.loc[0, 'actual_delivery_time']), df.loc[0, 'actual_delivery_time'])
print(df['actual_delivery_time'].head(5).tolist())

<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2015-02-06 23:27:16+00:00
[Timestamp('2015-02-06 23:27:16+0000', tz='UTC'), Timestamp('2015-02-10 22:56:29+0000', tz='UTC'), Timestamp('2015-01-22 21:09:09+0000', tz='UTC'), Timestamp('2015-02-03 22:13:00+0000', tz='UTC'), Timestamp('2015-02-15 03:20:26+0000', tz='UTC')]


In [175]:
df = df.copy()
df.columns = df.columns.str.strip()

df['created_at'] = pd.to_datetime(df['created_at'].astype(str).str.strip(), errors='coerce', format='mixed', utc=True)
df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'].astype(str).str.strip(), errors='coerce', format='mixed', utc=True)

print("created_at dtype:", df['created_at'].dtype)
print("actual_delivery_time dtype:", df['actual_delivery_time'].dtype)

created_at dtype: datetime64[ns, UTC]
actual_delivery_time dtype: datetime64[ns, UTC]


In [176]:
#Evaluating actual delivery duration in minutes

diff = df['actual_delivery_time'] - df['created_at']

# force it into a real timedelta even if pandas thinks it's object
diff = pd.to_timedelta(diff, errors='coerce')

df['actual_delivery_minutes'] = diff / pd.Timedelta(minutes=1)

df = df.dropna(subset=['actual_delivery_minutes'])
df = df[df['actual_delivery_minutes'] >= 0]

In [177]:
# understanding the data 

df['actual_delivery_minutes'].describe()

count     10586.000000
mean         61.636271
std        1379.287043
min           3.716667
25%          35.487500
50%          44.933333
75%          57.200000
max      141947.650000
Name: actual_delivery_minutes, dtype: float64

In [178]:
###Based on the above results,few orders have broken timestamps which would destroy averages, correlations and dashboards if not handled well.
###Therefore for operational analysis, deliveries longer than a realistic (240 minutes - 4 hours) upperbound are traeted as anomalies and excluded.

In [179]:
df = df[df['actual_delivery_minutes'] <= 240]

In [180]:
df['actual_delivery_minutes'].describe()

count    10583.000000
mean        48.188050
std         18.439973
min          3.716667
25%         35.483333
50%         44.933333
75%         57.183333
max        217.266667
Name: actual_delivery_minutes, dtype: float64

In [181]:
df.columns

Index(['market_id', 'created_at', 'actual_delivery_time', 'store_id',
       'store_primary_category', 'order_protocol', 'total_items', 'subtotal',
       'num_distinct_items', 'min_item_price', 'max_item_price',
       'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration',
       'actual_delivery_minutes'],
      dtype='object')

In [182]:
#Expected Delivery Minutes

df.loc[:, 'expected_delivery_minutes'] = (
    df['estimated_order_place_duration'] +
    df['estimated_store_to_consumer_driving_duration']
) / 60

In [183]:
df['expected_delivery_minutes'].describe()

count    10559.000000
mean        14.308187
std          3.908660
min          4.733333
25%         11.550000
50%         14.200000
75%         17.016667
max         38.983333
Name: expected_delivery_minutes, dtype: float64

In [184]:
#defining late delivery flag

df.loc[:, 'late_delivery_flag']=(df['actual_delivery_minutes']>df['expected_delivery_minutes']).astype(int)

In [185]:
df['late_delivery_flag'].value_counts(normalize=True)

late_delivery_flag
1    0.997638
0    0.002362
Name: proportion, dtype: float64

In [186]:
# Analyzing Market-Level Perfermonace

# Group by market
market_perf = df.groupby('market_id').agg(
    order_count=('store_id', 'count'),
    median_delivery_minutes=('actual_delivery_minutes', 'median'),
    mean_delivery_minutes=('actual_delivery_minutes', 'mean'),
    std_delivery_minutes=('actual_delivery_minutes', 'std'),
    q25_delivery_minutes=('actual_delivery_minutes', lambda x: x.quantile(0.25)),
    q75_delivery_minutes=('actual_delivery_minutes', lambda x: x.quantile(0.75)),
    late_prop=('late_delivery_flag', 'mean')  # use only for relative comparison
).reset_index()

# Sort markets by median delivery time descending
market_perf = market_perf.sort_values(by='median_delivery_minutes', ascending=False)

market_perf.head(10)  # preview top 10 slowest markets


,market_id,order_count,median_delivery_minutes,mean_delivery_minutes,std_delivery_minutes,q25_delivery_minutes,q75_delivery_minutes,late_prop
0,1.0,2044,48.350000,52.100342,21.004243,37.162500,62.150000,0.999022
1,2.0,2509,45.400000,48.053926,17.049250,36.383333,56.866667,0.996413
5,6.0,712,44.641667,47.574743,17.343910,35.662500,55.983333,0.998596
2,3.0,1351,44.416667,48.286640,19.124789,35.666667,57.083333,0.997039
4,5.0,1005,43.500000,46.585887,17.373645,34.733333,54.600000,0.999005
3,4.0,2909,43.350000,46.183568,17.553111,34.083333,54.750000,0.997250


In [187]:
#Restaurant-Level Analysis

restaurant_perf = df.groupby(['store_id', 'store_primary_category']).agg(
    order_count=('actual_delivery_minutes', 'count'),
    median_delivery_minutes=('actual_delivery_minutes', 'median'),
    mean_delivery_minutes=('actual_delivery_minutes', 'mean'),
    std_delivery_minutes=('actual_delivery_minutes', 'std'),
    q25_delivery_minutes=('actual_delivery_minutes', lambda x: x.quantile(0.25)),
    q75_delivery_minutes=('actual_delivery_minutes', lambda x: x.quantile(0.75))
).reset_index()

# Sort top slowest restaurants by median delivery time
restaurant_perf.sort_values(by='median_delivery_minutes', ascending=False, inplace=True)

restaurant_perf.head(10)


,store_id,store_primary_category,order_count,median_delivery_minutes,mean_delivery_minutes,std_delivery_minutes,q25_delivery_minutes,q75_delivery_minutes
685,5847.0,seafood,1,136.200000,136.200000,NaN,136.200000,136.200000
567,4766.0,vietnamese,2,130.391667,130.391667,122.859803,86.954167,173.829167
818,6824.0,thai,1,122.833333,122.833333,NaN,122.833333,122.833333
330,2928.0,sandwich,1,111.033333,111.033333,NaN,111.033333,111.033333
410,3625.0,breakfast,1,110.283333,110.283333,NaN,110.283333,110.283333
90,1019.0,american,1,108.033333,108.033333,NaN,108.033333,108.033333
284,2711.0,convenience-store,2,106.558333,106.558333,20.918576,99.162500,113.954167
25,199.0,hawaiian,1,98.733333,98.733333,NaN,98.733333,98.733333
501,4270.0,mexican,1,98.500000,98.500000,NaN,98.500000,98.500000
734,6216.0,thai,1,98.450000,98.450000,NaN,98.450000,98.450000


In [188]:
#Dasher Supply vs Demand Analysis

df.loc[:, 'congestion_ratio'] = df['total_busy_dashers'] / df['total_onshift_dashers']

dasher_perf = df.groupby('market_id').agg(
    avg_congestion=('congestion_ratio', 'mean'),
    median_delivery=('actual_delivery_minutes', 'median')
).reset_index()

dasher_perf.head(10)

,market_id,avg_congestion,median_delivery
0,1.0,0.958205,48.350000
1,2.0,0.961419,45.400000
2,3.0,inf,44.416667
3,4.0,0.986697,43.350000
4,5.0,0.864910,43.500000
5,6.0,1.021423,44.641667


In [189]:
# Time-of-Day & Peak Hour Analysis

df.loc[:, 'order_hour'] = df['created_at'].dt.hour

hourly_perf = df.groupby('order_hour').agg(
    order_count=('store_id', 'count'),
    median_delivery_minutes=('actual_delivery_minutes', 'median'),
    mean_delivery_minutes=('actual_delivery_minutes', 'mean')
).reset_index()

hourly_perf.head(10)

,order_hour,order_count,median_delivery_minutes,mean_delivery_minutes
0,0,698,42.000000,44.726146
1,1,1466,46.983333,49.700864
2,2,1929,52.083333,55.367703
3,3,1468,50.708333,54.806698
4,4,845,41.416667,44.779369
5,5,404,35.366667,39.005569
6,6,95,36.416667,38.458596
7,14,1,72.766667,72.766667
8,15,24,41.858333,43.488194
9,16,133,43.016667,47.331328


In [190]:
# Analyze distance & order complexity impact to determine long-distance orders or complex orders take longer

complexity_perf = df.groupby(['total_items', 'num_distinct_items']).agg(
    median_delivery_minutes=('actual_delivery_minutes', 'median'),
    mean_delivery_minutes=('actual_delivery_minutes', 'mean'),
    order_count=('store_id', 'count')
).reset_index()

complexity_perf.head(10)

,total_items,num_distinct_items,median_delivery_minutes,mean_delivery_minutes,order_count
0,1.0,1.0,41.233333,44.564287,1975
1,2.0,1.0,44.033333,46.239578,363
2,2.0,2.0,43.283333,46.750007,2518
3,3.0,1.0,49.133333,53.422013,53
4,3.0,2.0,45.991667,48.873855,422
5,3.0,3.0,44.850000,47.932445,1669
6,4.0,1.0,41.166667,49.461404,19
7,4.0,2.0,43.583333,48.660731,146
8,4.0,3.0,45.933333,49.649046,367
9,4.0,4.0,46.983333,49.836047,872


In [191]:
market_perf.to_csv('market_perf.csv', index=False)

In [192]:
restaurant_perf.to_csv('restaurant_perf.csv', index=False)

In [193]:
dasher_perf.to_csv('dasher_perf.csv', index=False)

In [194]:
hourly_perf.to_csv('hourly_perf.csv', index=False)

In [195]:
complexity_perf.to_csv('complexity_perf.csv', index=False)

In [196]:
import duckdb

con = duckdb.connect(database='doordash.duckdb')
con.register('orders', df)

In [197]:
con.execute("""
CREATE OR REPLACE TABLE sla_performance AS
SELECT
    COUNT(*) AS total_orders,
    AVG(actual_delivery_minutes) AS avg_actual_minutes,
    AVG(expected_delivery_minutes) AS avg_expected_minutes,
    AVG(late_delivery_flag) AS late_rate
FROM orders
""")


In [198]:
con.execute("SELECT * FROM sla_performance").df()

,total_orders,avg_actual_minutes,avg_expected_minutes,late_rate
0,10583,48.18805,14.308187,0.997638


In [199]:
con.execute("""
CREATE OR REPLACE TABLE market_performance_sql AS
SELECT
    market_id,
    COUNT(*) AS order_count,
    MEDIAN(actual_delivery_minutes) AS median_delivery_minutes,
    AVG(late_delivery_flag) AS late_rate
FROM orders
GROUP BY market_id
""")

In [200]:
con.execute("SELECT * FROM market_performance_sql ORDER BY late_rate DESC").df()

,market_id,order_count,median_delivery_minutes,late_rate
0,NaN,53,47.500000,1.000000
1,1.0,2044,48.350000,0.999022
2,5.0,1005,43.500000,0.999005
3,6.0,712,44.641667,0.998596
4,4.0,2909,43.350000,0.997250
5,3.0,1351,44.416667,0.997039
6,2.0,2509,45.400000,0.996413


In [201]:
con.execute("""
CREATE OR REPLACE TABLE restaurant_risk AS
SELECT
    store_id,
    store_primary_category,
    COUNT(*) AS order_count,
    MEDIAN(actual_delivery_minutes) AS median_delivery_minutes,
    AVG(late_delivery_flag) AS late_rate
FROM orders
GROUP BY store_id, store_primary_category
HAVING COUNT(*) >= 20
""")


In [202]:
con.execute("""
SELECT * FROM restaurant_risk
ORDER BY late_rate DESC, median_delivery_minutes DESC
LIMIT 10
""").df()

,store_id,store_primary_category,order_count,median_delivery_minutes,late_rate
0,429.0,chinese,20,80.516667,1.0
1,90.0,american,26,63.816667,1.0
2,5847.0,italian,28,63.133333,1.0
3,1997.0,indian,86,57.475000,1.0
4,703.0,greek,44,55.933333,1.0
5,2618.0,fast,21,55.483333,1.0
6,2549.0,sandwich,81,55.083333,1.0
7,6172.0,sandwich,48,54.541667,1.0
8,6570.0,burger,95,54.366667,1.0
9,2938.0,pizza,120,54.041667,1.0


In [203]:
con.execute("""
CREATE OR REPLACE TABLE dasher_supply_demand AS
SELECT
    market_id,
    AVG(total_busy_dashers * 1.0 / NULLIF(total_onshift_dashers, 0)) AS avg_congestion,
    MEDIAN(actual_delivery_minutes) AS median_delivery_minutes
FROM orders
GROUP BY market_id
""")

In [204]:
con.execute("SELECT * FROM dasher_supply_demand").df()

,market_id,avg_congestion,median_delivery_minutes
0,4.0,0.986697,43.350000
1,5.0,0.864910,43.500000
2,1.0,0.958205,48.350000
3,6.0,1.021423,44.641667
4,NaN,0.933854,47.500000
5,2.0,0.961419,45.400000
6,3.0,0.895932,44.416667
